In [169]:
import mujoco
import mujoco.viewer
from pathlib import Path
import numpy as np
import time

WIN_MODEL_PATH = r"C:\Users\Daniel\Documents\Python-Projects\ocrl-piano\single_finger_free_trajectories\models\scene_right_piano_hand.xml"
UNIVERSAL_MODEL_PATH = Path(WIN_MODEL_PATH)

print(str(UNIVERSAL_MODEL_PATH))
model = mujoco.MjModel.from_xml_path(str(UNIVERSAL_MODEL_PATH))
data = mujoco.MjData(model)
mujoco.mj_forward(model, data)


C:\Users\Daniel\Documents\Python-Projects\ocrl-piano\single_finger_free_trajectories\models\scene_right_piano_hand.xml


In [17]:
end_effector_site_names = ["fingertip_index", "fingertip_middle"]


end_effector_site_ids = [mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_SITE, name) for name in end_effector_site_names]
print(end_effector_site_ids)


jacp = np.zeros((3, model.nv), dtype=float) # translational Jacobian
jacr = np.zeros((3, model.nv), dtype=float) # translational Jacobian

jacp_combined = np.zeros((3*len(end_effector_site_ids), model.nv), dtype=float)   

for id in end_effector_site_ids:
    print("Site:", mujoco.mj_id2name(model, mujoco.mjtObj.mjOBJ_SITE, id))
    
    print("Translational Jacobian: ")
    mujoco.mj_jacSite(model, data, jacp, jacr, id)
    print(jacp)
    print(jacp.shape)
    print()





[1, 2]
Site: fingertip_index
Translational Jacobian: 
[[-2.22044605e-16  0.00000000e+00  1.00000000e+00 -3.30000000e-02
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00  1.99000000e-01
   0.00000000e+00 -7.00000000e-02  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-1.00000000e+00  0.00000000e+00 -2.22044605e-16  0

In [165]:
def get_multifinger_jacobian(model, data, site_names):
    jacp = np.zeros((3, model.nv))
    jacr = np.zeros((3, model.nv))
    
    jac_combined = np.zeros((6*len(site_names), model.nv))
    
    for i, site_name in enumerate(site_names):
        mujoco.mj_jacSite(model, data, jacp, jacr, mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_SITE, site_name))
        # jac_combined[6*i:6*(i+1), :] = np.vstack((jacp, jacr))
        
        jac_combined[6*i:6*(i+1), :] = np.vstack((jacp, jacr))
    
    return jac_combined

def get_error(model, data, site_names, target_names):
    
    error = np.zeros(6*len(site_names)) # [site_1_pos_error, site_1_rot_error, ..., site_N_pos_error, site_N_rot_error]
    
    for i, site_name in enumerate(site_names):
        site_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_SITE, site_name)
        target_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_GEOM, target_names[i])
        
        # preallocate arrays for a single site computation
        error_pos = error[6*i:6*i+3]
        error_rot = error[6*i+3:6*(i+1)]

        site_quat = np.zeros(4)
        site_quat_conj = np.zeros(4)
        
        target_quat = np.zeros(4)
        
        error_quat = np.zeros(4)
        
        # positional error
        error_pos[:] = data.geom(target_id).xpos - data.site(site_id).xpos

        
        # orientation error
        mujoco.mju_mat2Quat(site_quat, data.site(site_id).xmat)
        mujoco.mju_mat2Quat(target_quat, data.geom(target_id).xmat)
        
        mujoco.mju_negQuat(site_quat_conj, site_quat)
        mujoco.mju_mulQuat(error_quat, target_quat, site_quat_conj)
        mujoco.mju_quat2Vel(error_rot, error_quat, 1.0)
        # print(np.hstack((error_pos, error_rot)))
        # error[6*i:6*(i+1)] = np.hstack((error_pos, error_rot))
        
    return error

def solve_multifinger_ik(model, data, site_names, target_names, step_size=0.5, tol=1e-3, max_iter=1000):
    """Returns the joint angles that place the fingertips (named by site_names) at the target poses.
    Args:
        model (_type_): _description_
        data (_type_): _description_
        site_names (_type_): _description_
        target_poses (_type_): _description_

    Returns:
        _type_: _description_
    """
    assert len(site_names) == len(target_names)
    
    jac = np.zeros((6, model.nv)) 
    
    # compute error
    error = get_error(model, data, site_names, target_names)
    
    # TODO: write function for error
    # solution loop
    iter = 0
    while (np.linalg.norm(error) > tol and iter<=max_iter):

        jac = get_multifinger_jacobian(model, data, site_names)
        dq = np.linalg.pinv(jac) @ error
        data.qpos += step_size*dq
        mujoco.mj_forward(model, data)
        iter+=1
        
       # compute error
        error = get_error(model, data, site_names, target_names)
        print(np.linalg.norm(error))
        

    
    return data.qpos, np.linalg.norm(error), iter

In [173]:
# end_effector_site_names = ["fingertip_index", "fingertip_middle"]
# target_names = ["keymarker_1", "keymarker_2"]
end_effector_site_names = ["fingertip_index"]
target_names = ["keymarker_1"]

# target_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_GEOM, target_names[0])
# print(target_id)

qpos, error, iter = solve_multifinger_ik(model, data, end_effector_site_names, target_names)
data.qpos = qpos
mujoco.mj_forward(model, data)
mujoco.viewer.launch(model, data)
print(iter)

1.0691328334811818
0.5507225736565632
0.2778456838690351
0.1392628256768056
0.06967221744754898
0.0348400779299632
0.017420211834964106
0.0087100444689765
0.0043549936609475004
0.0021774880280530846
0.001088741605579423
0.0005443701746692196
12


In [71]:
model.geom("keymarker_1").pos
target_poses = np.array([model.geom(name).pos for name in target_names])
print(target_poses)
model.geom("keymarker_2")
# data.geom("keymarker_1")


[[ 0.4601 -0.43    0.12  ]
 [ 0.3701  0.3     0.14  ]]


<_MjDataGeomViews
  id: 50
  name: ''
  xmat: array([-1.18936541e-02, -7.13205759e-06,  9.99929268e-01,  1.98278251e-04,
       -9.99999980e-01, -4.77414220e-06,  9.99929248e-01,  1.98207445e-04,
        1.18936553e-02])
  xpos: array([ 0.35639065, -0.03299879,  0.24000976])
>